<a href="https://colab.research.google.com/github/PLEX-GR00T/Maze_solving_MDP/blob/main/Q_learning_(Taxi_v3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
# reset the environment to a new, random state
env.reset() 
env.render()
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [3]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [14]:
import numpy
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [15]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    #TODO: implement the Q-learning algorithm
    while not done:
      if random.uniform(0,1) < epsilon:
        action = env.action_space.sample()
      else:
        action = numpy.argmax(q_table[state])

      nxtState, reward, done, info = new = env.step(action)

      oldValue =  q_table[state,action]
      nxtMax = numpy.max(q_table[nxtState])


      newValue = (1-alpha) * oldValue + alpha * (reward + gamma * nxtMax)
      q_table[state, action] = newValue

      if reward == -10:
        penalties += 1
      
      state = nxtState

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 7s, sys: 14.2 s, total: 1min 21s
Wall time: 1min 9s


In [16]:
q_table[328]

array([ -2.4131659 ,  -2.27325184,  -2.40018244,  -2.3604151 ,
       -10.79628063, -10.67814863])

In [17]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.93
Average penalties per episode: 0.0
